In [1]:
import numpy as np
import librosa
import os
import glob
import pandas as pd
import pickle

In [4]:
audio = '/home/ubuntu/UrbanSound8K/audio'
folders = ["fold1","fold2","fold3","fold4","fold5","fold6","fold7","fold8","fold9","fold10"]

In [1]:
def get_audio_features(parent_dir, sub_dirs, file_ext="*.wav"):
    features = np.empty((0,66))
    labels, folds = [], []
    for idx, sub_dir in enumerate(sub_dirs):
        print(f"Processing {sub_dir}...")
        for file in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            librosa_load, librosa_sampling_rate = librosa.load(file, mono=True)
            stft = np.abs(librosa.stft(librosa_load))
            mfcc_mean = np.mean(librosa.feature.mfcc(y=librosa_load, sr=librosa_sampling_rate, n_mfcc=13).T,axis=0)
            mfcc_std = np.std(librosa.feature.mfcc(y=librosa_load, sr=librosa_sampling_rate, n_mfcc=13).T,axis=0)
            contrast_mean = np.mean(librosa.feature.spectral_contrast(S=stft, sr=librosa_sampling_rate).T,axis=0)
            contrast_std = np.std(librosa.feature.spectral_contrast(S=stft, sr=librosa_sampling_rate).T,axis=0)
            chroma_mean = np.mean(librosa.feature.chroma_stft(S=stft, sr=librosa_sampling_rate).T,axis=0)
            chroma_std = np.std(librosa.feature.chroma_stft(S=stft, sr=librosa_sampling_rate).T,axis=0)
            rolloff_mean = np.mean(librosa.feature.spectral_rolloff(S=stft, sr=librosa_sampling_rate).T,axis=0)
            rolloff_std = np.std(librosa.feature.spectral_rolloff(S=stft, sr=librosa_sampling_rate).T,axis=0)
            feature_cols = np.hstack([mfcc_mean, mfcc_std, contrast_mean, contrast_std, 
                                      chroma_mean, chroma_std, rolloff_mean, rolloff_std])
            features = np.vstack((features, feature_cols))
            target = (file.split('/')[6].split('-')[1])
            labels.append(int(target))
            folds.append(sub_dir)
    return features, labels, folds

In [8]:
%%capture
features, labels, folds = get_audio_features(audio, folders)

In [10]:
np.savetxt("features.csv", features, delimiter=",")
with open('labels.pkl', 'wb') as f:
    pickle.dump(labels_ps, f)
with open('folds.pkl', 'wb') as f:
    pickle.dump(folds_ps, f)

In [74]:
# with open('folds.pkl', 'rb') as f:
#     folds = pickle.load(f)

In [76]:
# with open('labels.pkl', 'rb') as f:
#     labels = pickle.load(f)

In [12]:
col_list = []
for i in range(1, 14):
    col_list.append('mfcc_mean' + str(i))
for i in range(1, 14):
    col_list.append('mfcc_std' + str(i))
for i in range(1, 8):
    col_list.append('contrast_mean' + str(i))
for i in range(1, 8):
    col_list.append('contrast_std' + str(i))
for i in range(1, 13):
    col_list.append('chroma_mean' + str(i))
for i in range(1, 13):
    col_list.append('chroma_std' + str(i))
col_list.append('rolloff_mean')
col_list.append('rolloff_std')

In [13]:
len(col_list)

66

In [14]:
df = pd.read_csv('./features.csv', names=col_list)

In [16]:
df.head()

,mfcc_mean1,mfcc_mean2,mfcc_mean3,mfcc_mean4,mfcc_mean5,mfcc_mean6,mfcc_mean7,mfcc_mean8,mfcc_mean9,mfcc_mean10,...,chroma_std5,chroma_std6,chroma_std7,chroma_std8,chroma_std9,chroma_std10,chroma_std11,chroma_std12,rolloff_mean,rolloff_std
0,-134.711482,29.417638,-92.563787,42.631234,-31.690849,3.436242,-23.509194,4.225683,-17.788425,1.603156,...,0.103158,0.062446,0.080010,0.109556,0.116632,0.117497,0.107907,0.084180,5285.834704,948.935306
1,-238.800392,99.541359,-30.091488,27.989554,-4.310405,23.445616,16.906146,13.645994,2.501890,-2.352757,...,0.210673,0.204993,0.253155,0.205989,0.227027,0.197083,0.201994,0.252197,4340.060654,637.735627
2,-90.385250,57.995826,-72.200805,-1.476553,-23.419294,-12.040609,-20.352058,-13.862473,-15.402611,-1.148808,...,0.115999,0.122231,0.141606,0.131073,0.126662,0.115871,0.128805,0.137801,4752.987830,632.733050
3,-392.359673,58.827135,13.025519,27.993198,-10.317400,-7.273219,-6.759042,-1.250139,0.107338,10.704278,...,0.256085,0.277229,0.263273,0.251291,0.246304,0.248324,0.251200,0.240325,5368.862333,1181.635958
4,-109.480069,100.928565,-36.085947,19.378924,1.225659,4.803250,1.805358,0.049484,0.404032,-5.566660,...,0.118850,0.113725,0.114777,0.115476,0.112360,0.110592,0.126750,0.106746,4215.404568,363.744472


In [17]:
sr_labels = pd.Series(labels)

In [18]:
df['label'] = sr_labels

In [19]:
sr_folds = pd.Series(folds)

In [20]:
df['fold'] = sr_folds

In [21]:
df.head()

,mfcc_mean1,mfcc_mean2,mfcc_mean3,mfcc_mean4,mfcc_mean5,mfcc_mean6,mfcc_mean7,mfcc_mean8,mfcc_mean9,mfcc_mean10,...,chroma_std7,chroma_std8,chroma_std9,chroma_std10,chroma_std11,chroma_std12,rolloff_mean,rolloff_std,label,fold
0,-134.711482,29.417638,-92.563787,42.631234,-31.690849,3.436242,-23.509194,4.225683,-17.788425,1.603156,...,0.080010,0.109556,0.116632,0.117497,0.107907,0.084180,5285.834704,948.935306,6,fold1
1,-238.800392,99.541359,-30.091488,27.989554,-4.310405,23.445616,16.906146,13.645994,2.501890,-2.352757,...,0.253155,0.205989,0.227027,0.197083,0.201994,0.252197,4340.060654,637.735627,2,fold1
2,-90.385250,57.995826,-72.200805,-1.476553,-23.419294,-12.040609,-20.352058,-13.862473,-15.402611,-1.148808,...,0.141606,0.131073,0.126662,0.115871,0.128805,0.137801,4752.987830,632.733050,4,fold1
3,-392.359673,58.827135,13.025519,27.993198,-10.317400,-7.273219,-6.759042,-1.250139,0.107338,10.704278,...,0.263273,0.251291,0.246304,0.248324,0.251200,0.240325,5368.862333,1181.635958,2,fold1
4,-109.480069,100.928565,-36.085947,19.378924,1.225659,4.803250,1.805358,0.049484,0.404032,-5.566660,...,0.114777,0.115476,0.112360,0.110592,0.126750,0.106746,4215.404568,363.744472,7,fold1


In [22]:
with open('raw_df.pkl', 'wb') as f:
    pickle.dump(df, f)